# CSP Solution Validator - Version Simplifiée

Ce notebook valide les solutions CSP en vérifiant qu'elles satisfont toutes les contraintes.

## Fonctionnalités
- Charge une instance CSP et ses solutions correspondantes
- Vérifie que les solutions satisfont toutes les contraintes
- Signale les contraintes violées (le cas échéant)
- Support pour les solutions régulières et Gurobi
- Validation automatique de toutes les instances

## Usage

```python
# Valider une instance spécifique
validate_instance('equality_example')

# Valider toutes les instances
validate_all_instances()
```


In [1]:
# Import des modules requis
import os
import glob
from typing import List, Tuple, Dict, Set
from dataclasses import dataclass

## Classes et Fonctions de Parsing


In [2]:
@dataclass
class CSPInstance:
    """Représente une instance CSP avec variables, domaines et contraintes."""
    num_variables: int
    domains: Dict[int, Tuple[int, int]]  # var_id -> (min, max)
    constraints: List[Tuple[int, int, Set[Tuple[int, int]]]]  # (var1, var2, allowed_pairs)
    
    def is_valid_assignment(self, var_id: int, value: int) -> bool:
        """Vérifie si une valeur est valide pour une variable."""
        min_val, max_val = self.domains[var_id]
        return min_val <= value <= max_val

@dataclass
class ValidationResult:
    """Résultat de la validation d'une solution."""
    is_valid: bool
    violated_constraints: List[Tuple[int, int, str]]
    domain_violations: List[Tuple[int, int, str]]
    missing_variables: List[int]
    extra_variables: List[int]


In [3]:
def parse_csp_instance(filename: str) -> CSPInstance:
    """Parse un fichier d'instance CSP."""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip() and not line.strip().startswith('#')]
    
    num_variables = int(lines[0])
    
    # Parse domains
    domains = {}
    line_idx = 1
    for i in range(num_variables):
        parts = lines[line_idx].split()
        var_id = int(parts[0])
        min_val = int(parts[1])
        max_val = int(parts[2])
        domains[var_id] = (min_val, max_val)
        line_idx += 1
    
    # Parse constraints
    num_constraints = int(lines[line_idx])
    line_idx += 1
    constraints = []
    
    for i in range(num_constraints):
        line = lines[line_idx]
        parts = line.split()
        var1 = int(parts[0])
        var2 = int(parts[1])
        
        allowed_pairs = set()
        for part in parts[2:]:
            if part.startswith('(') and part.endswith(')'):
                pair_str = part[1:-1]
                val1, val2 = map(int, pair_str.split(','))
                allowed_pairs.add((val1, val2))
        
        constraints.append((var1, var2, allowed_pairs))
        line_idx += 1
    
    return CSPInstance(num_variables, domains, constraints)

def parse_solution_file(filename: str) -> List[Dict[int, int]]:
    """Parse un fichier de solution et retourne la liste des solutions."""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    solutions = []
    for line in lines:
        if line.startswith('#') or not line or line == 'NO SOLUTION':
            continue
        
        # Parse solution line like "0=1 1=2 2=2"
        solution = {}
        assignments = line.split()
        for assignment in assignments:
            if '=' in assignment:
                var_str, val_str = assignment.split('=')
                var_id = int(var_str)
                value = int(val_str)
                solution[var_id] = value
        
        if solution:
            solutions.append(solution)
    
    return solutions

## Fonction de Validation Principale


In [4]:
def validate_single_solution(instance: CSPInstance, solution: Dict[int, int]) -> ValidationResult:
    """Valide une solution unique contre une instance."""
    violated_constraints = []
    domain_violations = []
    missing_variables = []
    extra_variables = []
    
    # Vérifier les violations de domaine
    for var_id, value in solution.items():
        if not instance.is_valid_assignment(var_id, value):
            domain_violations.append((var_id, value, f"Valeur {value} pas dans le domaine {instance.domains[var_id]}"))
    
    # Vérifier les variables manquantes
    for var_id in range(instance.num_variables):
        if var_id not in solution:
            missing_variables.append(var_id)
    
    # Vérifier les variables supplémentaires
    for var_id in solution.keys():
        if var_id >= instance.num_variables:
            extra_variables.append(var_id)
    
    # Vérifier les violations de contraintes
    for var1, var2, allowed_pairs in instance.constraints:
        if var1 in solution and var2 in solution:
            val1 = solution[var1]
            val2 = solution[var2]
            if (val1, val2) not in allowed_pairs:
                violated_constraints.append((var1, var2, f"Paire ({val1}, {val2}) non autorisée"))
    
    is_valid = not (violated_constraints or domain_violations or missing_variables or extra_variables)
    
    return ValidationResult(
        is_valid=is_valid,
        violated_constraints=violated_constraints,
        domain_violations=domain_violations,
        missing_variables=missing_variables,
        extra_variables=extra_variables
    )

def validate_instance(instance_name: str, instances_dir: str = "../instances/instances", solutions_dir: str = "../solutions/solutions"):
    """Valide une instance CSP spécifique."""
    print(f"\n{'='*60}")
    print(f"VALIDATION DE L'INSTANCE: {instance_name}")
    print(f"{'='*60}")
    
    # Charger l'instance
    instance_file = os.path.join(instances_dir, f"{instance_name}.csp")
    if not os.path.exists(instance_file):
        print(f"❌ Fichier d'instance non trouvé: {instance_file}")
        return
    
    instance = parse_csp_instance(instance_file)
    print(f"✅ Instance chargée: {instance.num_variables} variables, {len(instance.constraints)} contraintes")
    
    # Vérifier les solutions régulières
    regular_file = os.path.join(solutions_dir, f"{instance_name}.sol")
    if os.path.exists(regular_file):
        print(f"\n🔍 Solutions régulières ({regular_file}):")
        solutions = parse_solution_file(regular_file)
        if solutions:
            for i, solution in enumerate(solutions):
                result = validate_single_solution(instance, solution)
                print(f"  Solution {i+1}: {'✅ VALIDE' if result.is_valid else '❌ INVALIDE'}")
                if not result.is_valid:
                    if result.domain_violations:
                        print(f"    Violations de domaine: {len(result.domain_violations)}")
                    if result.missing_variables:
                        print(f"    Variables manquantes: {result.missing_variables}")
                    if result.extra_variables:
                        print(f"    Variables supplémentaires: {result.extra_variables}")
                    if result.violated_constraints:
                        print(f"    Violations de contraintes: {len(result.violated_constraints)}")
        else:
            print("  Aucune solution trouvée")
    else:
        print(f"⚠️  Fichier de solutions régulières non trouvé: {regular_file}")
    
    # Vérifier les solutions Gurobi
    gurobi_file = os.path.join(solutions_dir, f"{instance_name}_gb.sol")
    if os.path.exists(gurobi_file):
        print(f"\n🔍 Solutions Gurobi ({gurobi_file}):")
        solutions = parse_solution_file(gurobi_file)
        if solutions:
            for i, solution in enumerate(solutions):
                result = validate_single_solution(instance, solution)
                print(f"  Solution {i+1}: {'✅ VALIDE' if result.is_valid else '❌ INVALIDE'}")
                if not result.is_valid:
                    if result.domain_violations:
                        print(f"    Violations de domaine: {len(result.domain_violations)}")
                    if result.missing_variables:
                        print(f"    Variables manquantes: {result.missing_variables}")
                    if result.extra_variables:
                        print(f"    Variables supplémentaires: {result.extra_variables}")
                    if result.violated_constraints:
                        print(f"    Violations de contraintes: {len(result.violated_constraints)}")
        else:
            print("  Aucune solution trouvée")
    else:
        print(f"⚠️  Fichier de solutions Gurobi non trouvé: {gurobi_file}")

In [5]:
def validate_all_instances(instances_dir: str = "../instances/instances", solutions_dir: str = "../solutions/solutions"):
    """Valide toutes les instances dans le dossier d'instances."""
    print("="*80)
    print("VALIDATION DE TOUTES LES INSTANCES")
    print("="*80)
    
    if not os.path.exists(instances_dir):
        print(f"❌ Dossier d'instances non trouvé: {instances_dir}")
        return
    
    # Trouver tous les fichiers .csp
    csp_files = glob.glob(os.path.join(instances_dir, "*.csp"))
    
    if not csp_files:
        print("❌ Aucun fichier .csp trouvé dans le dossier d'instances")
        return
    
    print(f"📁 {len(csp_files)} instances trouvées")
    
    # Valider chaque instance
    for csp_file in sorted(csp_files):
        instance_name = os.path.splitext(os.path.basename(csp_file))[0]
        validate_instance(instance_name, instances_dir, solutions_dir)
    
    print("\n" + "="*80)
    print("FIN DE LA VALIDATION")
    print("="*80)


## Exemples d'Utilisation


In [6]:
# Exemple 1: Valider une instance spécifique
print("Exemple 1: Validation d'une instance spécifique")
validate_instance('equality_example')

Exemple 1: Validation d'une instance spécifique

VALIDATION DE L'INSTANCE: equality_example
✅ Instance chargée: 4 variables, 4 contraintes

🔍 Solutions régulières (../solutions/solutions/equality_example.sol):
  Solution 1: ✅ VALIDE
  Solution 2: ✅ VALIDE
  Solution 3: ✅ VALIDE

🔍 Solutions Gurobi (../solutions/solutions/equality_example_gb.sol):
  Solution 1: ✅ VALIDE
  Solution 2: ✅ VALIDE
  Solution 3: ✅ VALIDE


## Validation de Toutes les Instances


In [7]:
# Exemple 2: Valider toutes les instances
print("Exemple 2: Validation de toutes les instances")
validate_all_instances()

Exemple 2: Validation de toutes les instances
VALIDATION DE TOUTES LES INSTANCES
📁 30 instances trouvées

VALIDATION DE L'INSTANCE: equality_example
✅ Instance chargée: 4 variables, 4 contraintes

🔍 Solutions régulières (../solutions/solutions/equality_example.sol):
  Solution 1: ✅ VALIDE
  Solution 2: ✅ VALIDE
  Solution 3: ✅ VALIDE

🔍 Solutions Gurobi (../solutions/solutions/equality_example_gb.sol):
  Solution 1: ✅ VALIDE
  Solution 2: ✅ VALIDE
  Solution 3: ✅ VALIDE

VALIDATION DE L'INSTANCE: example_inequality
✅ Instance chargée: 4 variables, 6 contraintes

🔍 Solutions régulières (../solutions/solutions/example_inequality.sol):
  Aucune solution trouvée

🔍 Solutions Gurobi (../solutions/solutions/example_inequality_gb.sol):
  Aucune solution trouvée

VALIDATION DE L'INSTANCE: example_random
✅ Instance chargée: 4 variables, 6 contraintes

🔍 Solutions régulières (../solutions/solutions/example_random.sol):
  Solution 1: ✅ VALIDE
  Solution 2: ✅ VALIDE
  Solution 3: ✅ VALIDE
  Soluti